In [1]:
from mcap.reader import SeekingReader, make_reader
import io
from rclpy.serialization import deserialize_message
from sensor_msgs.msg import CompressedImage, CameraInfo
from PIL import Image
import numpy


In [2]:
path = "sensor_data.mcap"

In [3]:
reader = SeekingReader(open(path, "rb"))

In [15]:
reader.get_summary().schemas

{1: Schema(id=1, data=b'# This message defines meta information for a camera. It should be in a\n# camera namespace on topic "camera_info" and accompanied by up to five\n# image topics named:\n#\n#   image_raw - raw data from the camera driver, possibly Bayer encoded\n#   image            - monochrome, distorted\n#   image_color      - color, distorted\n#   image_rect       - monochrome, rectified\n#   image_rect_color - color, rectified\n#\n# The image_pipeline contains packages (image_proc, stereo_image_proc)\n# for producing the four processed image topics from image_raw and\n# camera_info. The meaning of the camera parameters are described in\n# detail at http://www.ros.org/wiki/image_pipeline/CameraInfo.\n#\n# The image_geometry package provides a user-friendly interface to\n# common operations using this meta information. If you want to, e.g.,\n# project a 3d point into image coordinates, we strongly recommend\n# using image_geometry.\n#\n# If the camera is uncalibrated, the matr

In [ ]:
topics = [reader.get_summary().channels[i].topic for i in range(1, 21)]
for i in sorted(topics):
    print(i)

/center_front/camera_info
/center_front/image_rect/compressed
/center_front/image_rect/compressedDepth
/center_rear/camera_info
/center_rear/image_rect/compressed
/center_rear/image_rect/compressedDepth
/driver_front/camera_info
/driver_front/image_rect/compressed
/driver_front/image_rect/compressedDepth
/driver_rear/camera_info
/driver_rear/image_rect/compressed
/driver_rear/image_rect/compressedDepth
/passenger_front/camera_info
/passenger_front/image_rect/compressed
/passenger_front/image_rect/compressedDepth
/passenger_rear/camera_info
/passenger_rear/image_rect/compressed
/passenger_rear/image_rect/compressedDepth
/sensing/lidar/front/pointcloud_raw_ex
/sensing/lidar/rear/pointcloud_raw_ex


In [ ]:
for schema, ch, msg in reader.iter_messages(topics="/center_front/camera_info"):
    msg : CameraInfo = deserialize_message(msg.data, CameraInfo)
    print(msg)
    break

sensor_msgs.msg.CameraInfo(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1758040178, nanosec=804728760), frame_id='center_front_cam/camera_link'), height=1080, width=1920, distortion_model='plumb_bob', d=[-0.3508414030075073, 0.10376661270856857, -0.0005301302298903465, 0.00696601951494813, 0.0], k=array([1.54580408e+03, 0.00000000e+00, 9.50406921e+02, 0.00000000e+00,
       1.55801074e+03, 4.36352997e+02, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00]), r=array([1., 0., 0., 0., 1., 0., 0., 0., 1.]), p=array([1.29559326e+03, 0.00000000e+00, 9.70977295e+02, 0.00000000e+00,
       0.00000000e+00, 1.47800549e+03, 4.25816864e+02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]), binning_x=0, binning_y=0, roi=sensor_msgs.msg.RegionOfInterest(x_offset=0, y_offset=0, height=0, width=0, do_rectify=False))


In [22]:
reader.get_summary().statistics

Statistics(attachment_count=0, channel_count=13, channel_message_counts={19: 183, 10: 183, 3: 184, 11: 183, 20: 184, 7: 184, 18: 184, 14: 184, 1: 184, 9: 199, 12: 185, 15: 184, 2: 184}, chunk_count=0, message_count=2405, message_end_time=1758040198571180536, message_start_time=1758040178687887576, metadata_count=1, schema_count=3)

In [ ]:
d = {}

for schema, ch, msg in reader.iter_messages(start_time=1758040178687887576, end_time=1758040198571180536):
    print(ch.topic)
    if ch.topic.endswith("compressed"):
        t = ch.topic.split("/")[-1]
        print(t)
        if t not in d:
            d[t] = 0

        msg : CompressedImage = deserialize_message(msg.data, CompressedImage)
        img = Image.open(io.BytesIO(msg.data))
        img.save(f"{t}-{d[t]}.jpeg")
        d[t] += 1
        print(d)

In [38]:
from collections import defaultdict
import math

start_time = 1758040178687887576
end_time   = 1758040198571180536
bin_size_ns = 1_000_000_000  # 1 second bins

# topic -> {bin_index -> count}
bins = defaultdict(lambda: defaultdict(int))

for schema, ch, msg in reader.iter_messages(start_time=start_time, end_time=end_time):
    if ch.topic.endswith("compressed"):
        # find which second this message belongs to
        sec = (msg.log_time - start_time) // bin_size_ns
        bins[ch.topic][sec] += 1

# Print per-second counts for each topic
for topic, sec_counts in bins.items():
    print(f"\nTopic: {topic}")
    for sec in sorted(sec_counts):
        t_abs = start_time/1e9 + sec   # convert to absolute seconds if you want
        count = sec_counts[sec]
        print(f"  second {sec:>4} (t≈{t_abs:.3f}s): {count} msg")


Topic: /driver_front/image_rect/compressed
  second    0 (t≈1758040178.688s): 9 msg
  second    1 (t≈1758040179.688s): 9 msg
  second    2 (t≈1758040180.688s): 9 msg
  second    3 (t≈1758040181.688s): 9 msg
  second    4 (t≈1758040182.688s): 9 msg
  second    5 (t≈1758040183.688s): 9 msg
  second    6 (t≈1758040184.688s): 10 msg
  second    7 (t≈1758040185.688s): 9 msg
  second    8 (t≈1758040186.688s): 9 msg
  second    9 (t≈1758040187.688s): 10 msg
  second   10 (t≈1758040188.688s): 9 msg
  second   11 (t≈1758040189.688s): 9 msg
  second   12 (t≈1758040190.688s): 9 msg
  second   13 (t≈1758040191.688s): 10 msg
  second   14 (t≈1758040192.688s): 8 msg
  second   15 (t≈1758040193.688s): 11 msg
  second   16 (t≈1758040194.688s): 10 msg
  second   17 (t≈1758040195.688s): 9 msg
  second   18 (t≈1758040196.688s): 10 msg
  second   19 (t≈1758040197.688s): 7 msg

Topic: /center_front/image_rect/compressed
  second    0 (t≈1758040178.688s): 9 msg
  second    1 (t≈1758040179.688s): 8 msg
  se

In [39]:
from collections import defaultdict

# Assuming you already have `reader` created
start_time = 1758040178687887576
end_time   = 1758040198571180536
BIN_NS = 1_000_000_000  # 1-second bins

# sec_index -> {"count": int, "bytes": int}
timeline = defaultdict(lambda: {"count": 0, "bytes": 0})

def human_bytes(n: int) -> str:
    for unit in ("B","KB","MB","GB","TB"):
        if n < 1024 or unit == "TB":
            return f"{n:.1f} {unit}"
        n /= 1024

for schema, ch, msg in reader.iter_messages(start_time=start_time, end_time=end_time):
    if not ch.topic.endswith("compressed"):
        continue
    sec = (msg.log_time - start_time) // BIN_NS
    # MCAP messages expose payload bytes as msg.data
    size = len(msg.data) if hasattr(msg, "data") and msg.data is not None else 0
    bucket = timeline[sec]
    bucket["count"] += 1
    bucket["bytes"] += size

# Print results: ith second from start -> total messages and total size
print(f"Per-second totals for topics *.compressed (start={start_time}, end={end_time})\n")
for sec in sorted(timeline):
    t_abs = start_time/1e9 + sec  # absolute seconds since epoch (approx)
    count = timeline[sec]["count"]
    total_bytes = timeline[sec]["bytes"]
    print(f"second {sec:>4} (t≈{t_abs:.3f}s): {count:>6} msgs, {total_bytes:>10} bytes ({human_bytes(total_bytes)})")


Per-second totals for topics *.compressed (start=1758040178687887576, end=1758040198571180536)

second    0 (t≈1758040178.688s):     51 msgs,   28490752 bytes (27.2 MB)
second    1 (t≈1758040179.688s):     51 msgs,   28386480 bytes (27.1 MB)
second    2 (t≈1758040180.688s):     57 msgs,   31735948 bytes (30.3 MB)
second    3 (t≈1758040181.688s):     53 msgs,   29521960 bytes (28.2 MB)
second    4 (t≈1758040182.688s):     56 msgs,   31091688 bytes (29.7 MB)
second    5 (t≈1758040183.688s):     55 msgs,   30824204 bytes (29.4 MB)
second    6 (t≈1758040184.688s):     57 msgs,   31894824 bytes (30.4 MB)
second    7 (t≈1758040185.688s):     56 msgs,   31364268 bytes (29.9 MB)
second    8 (t≈1758040186.688s):     55 msgs,   30551100 bytes (29.1 MB)
second    9 (t≈1758040187.688s):     56 msgs,   31268564 bytes (29.8 MB)
second   10 (t≈1758040188.688s):     54 msgs,   30110700 bytes (28.7 MB)
second   11 (t≈1758040189.688s):     54 msgs,   30120876 bytes (28.7 MB)
second   12 (t≈1758040190.68

In [ ]:
reader.get_summary().statistics.channel_count

{1: Channel(id=1, topic='/passenger_front/camera_info', message_encoding='cdr', metadata={'offered_qos_profiles': '- history: 1\n  depth: 10\n  reliability: 1\n  durability: 2\n  deadline:\n    sec: 9223372036\n    nsec: 854775807\n  lifespan:\n    sec: 9223372036\n    nsec: 854775807\n  liveliness: 1\n  liveliness_lease_duration:\n    sec: 9223372036\n    nsec: 854775807\n  avoid_ros_namespace_conventions: false'}, schema_id=1),
 2: Channel(id=2, topic='/driver_front/image_rect/compressed', message_encoding='cdr', metadata={'offered_qos_profiles': '- history: 1\n  depth: 10\n  reliability: 1\n  durability: 2\n  deadline:\n    sec: 9223372036\n    nsec: 854775807\n  lifespan:\n    sec: 9223372036\n    nsec: 854775807\n  liveliness: 1\n  liveliness_lease_duration:\n    sec: 9223372036\n    nsec: 854775807\n  avoid_ros_namespace_conventions: false'}, schema_id=2),
 3: Channel(id=3, topic='/driver_rear/image_rect/compressed', message_encoding='cdr', metadata={'offered_qos_profiles': '- hi

In [5]:
from collections import defaultdict
import cv2

# Assuming you already have `reader` created
start_time = 1758040178687887576
end_time   = 1758040198571180536
BIN_NS = 1_000_000_000  # 1-second bins

# sec_index -> {"count": int, "bytes": int}
timeline = defaultdict(lambda: {"count": 0, "bytes": 0})

def human_bytes(n: int) -> str:
    for unit in ("B","KB","MB","GB","TB"):
        if n < 1024 or unit == "TB":
            return f"{n:.1f} {unit}"
        n /= 1024

for schema, ch, msg in reader.iter_messages(start_time=start_time, end_time=end_time):
    if not ch.topic.endswith("compressed"):
        continue
    sec = (msg.log_time - start_time) // BIN_NS
    # MCAP messages expose payload bytes as msg.data
    compressed_image : CompressedImage = deserialize_message(msg.data, CompressedImage)
    arr = numpy.frombuffer(compressed_image.data, numpy.uint8)
    bgr = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    ok, encoded = cv2.imencode(".jpg", bgr, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

    size = encoded.size
    bucket = timeline[sec]
    bucket["count"] += 1
    bucket["bytes"] += size

# Print results: ith second from start -> total messages and total size
print(f"Per-second totals for topics *.compressed (start={start_time}, end={end_time})\n")
for sec in sorted(timeline):
    t_abs = start_time/1e9 + sec  # absolute seconds since epoch (approx)
    count = timeline[sec]["count"]
    total_bytes = timeline[sec]["bytes"]
    print(f"second {sec:>4} (t≈{t_abs:.3f}s): {count:>6} msgs, {total_bytes:>10} bytes ({human_bytes(total_bytes)})")


Per-second totals for topics *.compressed (start=1758040178687887576, end=1758040198571180536)

second    0 (t≈1758040178.688s):     51 msgs,   50643905 bytes (48.3 MB)
second    1 (t≈1758040179.688s):     51 msgs,   50484552 bytes (48.1 MB)
second    2 (t≈1758040180.688s):     57 msgs,   56444186 bytes (53.8 MB)
second    3 (t≈1758040181.688s):     53 msgs,   52520215 bytes (50.1 MB)
second    4 (t≈1758040182.688s):     56 msgs,   55327125 bytes (52.8 MB)
second    5 (t≈1758040183.688s):     55 msgs,   54782921 bytes (52.2 MB)
second    6 (t≈1758040184.688s):     57 msgs,   56699991 bytes (54.1 MB)
second    7 (t≈1758040185.688s):     56 msgs,   55734189 bytes (53.2 MB)
second    8 (t≈1758040186.688s):     55 msgs,   54376643 bytes (51.9 MB)
second    9 (t≈1758040187.688s):     56 msgs,   55597761 bytes (53.0 MB)
second   10 (t≈1758040188.688s):     54 msgs,   53561553 bytes (51.1 MB)
second   11 (t≈1758040189.688s):     54 msgs,   53568048 bytes (51.1 MB)
second   12 (t≈1758040190.68